In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Crawl stock price

## Selenium

In [2]:
apple_url = 'https://www.investing.com/equities/apple-computer-inc-historical-data'

In [3]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')
browser.maximize_window()

In [17]:
browser = webdriver.Firefox(executable_path='geckodriver.exe')

In [18]:
browser.get(apple_url)

### Close popup

In [ ]:
browser.find_element_by_class_name('popupCloseIcon.largeBannerCloser').click()

### Click the calendar

In [19]:
browser.find_element_by_id('widgetFieldDateRange').click()

### Find and input start date

In [21]:
startDate = browser.find_element_by_id('startDate')
startDate.clear()
startDate.send_keys('02/15/2001')
browser.find_element_by_id('applyBtn').send_keys(Keys.ENTER)

### Look up value

In [22]:
str_xpath = '/html/body/div[5]/section/div[9]/table[1]/tbody/tr[{}]/td[{}]'

In [23]:
date = []
close = []
Open = []
high = []
low = []

In [ ]:
%%time
for i in range(1, 5000+1):
    for j in range(1, 5+1):
        xpath = str_xpath.format(i, j)
        real_value = browser.find_element_by_xpath(xpath)
        if j == 1:
            date.append(real_value.text)
        if j == 2:
            close.append(real_value.text)
        if j == 3:
            Open.append(real_value.text)
        if j == 4:
            high.append(real_value.text)
        if j == 5:
            low.append(real_value.text)

In [25]:
len(date)

5000

In [26]:
len(close)

5000

In [27]:
len(Open)

5000

In [28]:
len(high)

5000

In [29]:
len(low)

5000

## Save dataframe

In [2]:
import pandas as pd

In [31]:
columns = ['Date', 'Close', 'Open', 'High', 'Low']
aapl_stock_price = pd.DataFrame({
    'Date': date, 
    'Close': close, 
    'Open': Open, 
    'High': high, 
    'Low': low
    })
aapl_stock_price

,Date,Close,Open,High,Low
0,"Dec 29, 2020",136.53,138.05,138.78,135.53
1,"Dec 28, 2020",136.69,133.99,137.34,133.52
2,"Dec 24, 2020",131.97,131.32,133.46,131.10
3,"Dec 23, 2020",130.96,132.18,132.32,130.83
4,"Dec 22, 2020",131.88,131.68,134.40,129.66
...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33


In [33]:
aapl_stock_price.to_csv('aapl_stock_price.csv', index=False)

In [3]:
aapl_df = pd.read_csv('aapl_stock_price.csv', index_col=False)

In [6]:
aapl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Date     5000 non-null object
Close    5000 non-null float64
Open     5000 non-null float64
High     5000 non-null float64
Low      5000 non-null float64
dtypes: float64(4), object(1)
memory usage: 195.4+ KB


In [41]:
aapl_df.describe()

,Close,Open,High,Low
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,19.715890,19.708146,19.917360,19.497556
std,24.018699,24.002924,24.293137,23.716058
min,0.230000,0.230000,0.240000,0.230000
25%,2.260000,2.277500,2.307500,2.240000
50%,11.830000,11.795000,11.920000,11.670000
75%,28.362500,28.352500,28.590000,28.082500
max,136.690000,138.050000,138.780000,135.530000


# Stock Indicators

## Trend Indicators 

### SMA
Simple moving average  
http://www.fmlabs.com/reference/default.htm?url=SimpleMA.htm

In [4]:
sma = aapl_df['Close'].rolling(window=3, min_periods=3).mean()
sma

0              NaN
1              NaN
2       135.063333
3       133.206667
4       131.603333
           ...    
4995      0.343333
4996      0.340000
4997      0.336667
4998      0.336667
4999      0.343333
Name: Close, Length: 5000, dtype: float64

In [7]:
aapl_df['SMA_3'] = sma
aapl_df

,Date,Close,Open,High,Low,SMA_3
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,NaN
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,NaN
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,135.063333
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,133.206667
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,131.603333
...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,0.343333
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,0.340000
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,0.336667
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,0.336667


In [9]:
aapl_df.to_csv('aapl_stock_price_with_sma.csv', index=False)

### EMA
Exponential moving average  
https://www.fmlabs.com/reference/default.htm?url=ExpMA.htm  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html

In [15]:
ema = aapl_df['Close'].ewm(span=3, adjust=False).mean()
ema

0       136.530000
1       136.610000
2       134.290000
3       132.625000
4       132.252500
           ...    
4995      0.341622
4996      0.340811
4997      0.335405
4998      0.337703
4999      0.348851
Name: Close, Length: 5000, dtype: float64

In [16]:
aapl_df['EMA_3'] = ema
aapl_df

,Date,Close,Open,High,Low,SMA_3,EMA_3
0,"Dec 29, 2020",136.53,138.05,138.78,135.53,136.530000,136.530000
1,"Dec 28, 2020",136.69,133.99,137.34,133.52,136.610000,136.610000
2,"Dec 24, 2020",131.97,131.32,133.46,131.10,135.063333,134.290000
3,"Dec 23, 2020",130.96,132.18,132.32,130.83,133.206667,132.625000
4,"Dec 22, 2020",131.88,131.68,134.40,129.66,131.603333,132.252500
...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",0.34,0.34,0.35,0.32,0.343333,0.341622
4996,"Feb 21, 2001",0.34,0.33,0.36,0.33,0.340000,0.340811
4997,"Feb 20, 2001",0.33,0.34,0.35,0.32,0.336667,0.335405
4998,"Feb 16, 2001",0.34,0.34,0.35,0.33,0.336667,0.337703


## Mean Reversion Indicators

### Bollinger Bands

## Volume Indicators

### OBV
https://www.fmlabs.com/reference/default.htm?url=OBV.htm  
https://stackoverflow.com/a/60093990/9480586  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L417  
OBV cần thông tin volume nữa

## Momentum Indicators

### MACD
Moving Average Convergence Divergence  
https://www.fmlabs.com/reference/default.htm?url=MACD.htm  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L219

## Relative Strength Indicators

### STOCH
Stochastic Oscillator

### RSI
Relative Strength Index  
https://www.fmlabs.com/reference/default.htm?url=RSI.htm  
https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py#L313